### Recursion

In [8]:
import random

In [1]:
# Solve a sum of array using recursion

def sum_array_recursion(array):
  if len(array)!=0:
    return sum_array_recursion(array[1:])+array[0] #<-recursion case
  elif len(array)==0:
    return 0 #<-base case

In [4]:
sum_array_recursion(range(10000))

RecursionError: maximum recursion depth exceeded

The error you are seeing, RecursionError: maximum recursion depth exceeded, happens because your recursive function sum_array_recursion is being called too many times. Python has a limit on how deep recursion can go to prevent infinite loops and stack overflow. When you pass a range object with 10**10 elements, the function tries to call itself that many times, quickly exceeding this limit.

For summing large arrays, an iterative approach or using the built-in sum() function is much more efficient and avoids this recursion depth issue.


I know the iterative version, but just curious how is the python built in sum() designed? If you can show me
That's a great question! Python's built-in sum() function is actually implemented in C, which makes it very efficient for summing iterables. It's designed to iterate over the elements and add them up, similar to the iterative approach we discussed.

Since I can't show you the C source code directly, here's a Python function that behaves similarly to the built-in sum(). It uses an iterative approach, which is how the sum() function works under the hood.

In [5]:
def sum_array_iterative(array):
    total=0
    for element in array:
        total+=element
    return total

In [6]:
sum_array_iterative(range(1000000000))

499999999500000000

### Quick Sort

In [7]:
# Divide and Conquer and then recursion
def quick_sort(array):
  if len(array)<2:
    return array
  else:
    pivot = array[0]
    sub_array_left = [i for i in array[1:] if i < pivot]
    sub_array_right = [i for i in array[1:] if i>= pivot]
    return quick_sort(sub_array_left) + [pivot] + quick_sort(sub_array_right)

In [13]:
import numpy as np

In [14]:
# Generate a list of n random integers without a for loop
n = 10**7 # Replace with the desired number of integers
random_integers = np.array(random.sample(range(10**8), n)) # Generates n unique integers between 0 and 100

In [15]:
sorted_array = quick_sort(random_integers)

In [ ]:
np_array = np.sort(random_integers)

Your recursive `quick_sort` implementation is much slower than `numpy.sort` for several reasons:

### 1. **Algorithm Efficiency**
- Your `quick_sort` uses Python lists and list comprehensions, which are slow for large arrays.
- Each recursion creates new lists (`sub_array_left`, `sub_array_right`), causing a lot of memory allocation and copying.
- `numpy.sort` uses highly optimized C code under the hood and operates directly on contiguous memory (NumPy arrays), making it much faster.

### 2. **Implementation Details**
- Your implementation is a simple, unoptimized version of quicksort (not in-place, always picks the first element as pivot).
- `numpy.sort` uses the **Timsort** algorithm for small arrays and **quicksort/mergesort/heapsort** (all optimized) for larger arrays, depending on the data type and options.

### 3. **Language Overhead**
- Python is an interpreted language and much slower for loops and recursion compared to compiled C code.
- NumPy leverages vectorized operations and compiled code, avoiding Python’s function call overhead.

---

### **Summary Table**

| Your `quick_sort`                | `numpy.sort`                        |
|----------------------------------|-------------------------------------|
| Pure Python, recursive           | C-optimized, vectorized             |
| Not in-place (creates new lists) | In-place (efficient memory use)     |
| Always picks first pivot         | Smart pivot selection               |
| Slow for large arrays            | Fast for large arrays               |

---

**In short:**  
`numpy.sort` is much faster because it uses optimized, compiled algorithms and efficient memory handling, while your Python quicksort is simple, recursive, and memory-inefficient. For large data, always use NumPy or built-in sorting functions.

## Hashing

### Hash Function and Hash Table
- Hash function maps a string to a integer ( typically an index of an array or list)
- It can be many-to-one or even one-to-one as well
- one-to-one or injective hash functions are good to have, but many times they are not available based on situations. That kind of situations are known as `collison`.
- A good hash function is which avoids collison.
- If a hash function faces collison, i.e. two different strings map to a same number, then sometimes it may create linked list to store data for those keys.
- Hash tables are the kind of Data Structures which stores these mapings of hash functions
- Python dictionary is a type of Hash table.
- Hash tables are good and fast at finding duplicate data
- Hash functions are good for caching data
- Hash tables typically stores data in arrays
- Load factor - (Number of blocks occupied)/(Total number of Blocks)
- When the load factor goes above 0.7, then the blocks need to be resized. Typically doubled up.

Yes 👍 — in CPython (the reference Python implementation), a **dictionary (`dict`) is implemented as a hash table** with open addressing and quadratic probing.

Now, about the **hash function** used:

* When you call `hash(obj)` in Python, it uses the built-in `hash()` function of that object.
* For built-in immutable types (`int`, `float`, `str`, `bytes`, `tuple`), CPython provides optimized hash implementations.
* For user-defined objects, the default is `id(obj)` unless you override `__hash__`.

---

### Hashing details by type (CPython):

* **Integers**: The hash is basically the integer value itself (clamped to machine word size).
* **Floats**: Same as hashing the equivalent integer for exact values, with special handling for `NaN`, `inf`, etc.
* **Strings**: A variant of **FNV-1a**–like algorithm (a simple multiplicative hash), randomized with a per-process seed since Python 3.3 to prevent hash-collision DoS attacks.

  * This is called **SipHash24** (cryptographically strong, introduced in Python 3.4+ as default for str/bytes).
* **Tuples**: Combined from the hash values of their elements using a rolling algorithm.

---

✅ So the short answer:

* Python dictionaries don’t use a single fixed hash function.
* They use the object’s `__hash__` result.
* For strings/bytes, CPython uses **SipHash24** internally.

Perfect 👍 Let me show you with Python code how **two different objects can have the same hash value** but still live correctly inside a dictionary:

```python
# Define a custom class
class AlwaysHash42:
    def __init__(self, value):
        self.value = value

    def __hash__(self):
        # Force all objects to have the same hash
        return 42

    def __eq__(self, other):
        return isinstance(other, AlwaysHash42) and self.value == other.value

    def __repr__(self):
        return f"Obj({self.value})"


# Create objects with different values
a = AlwaysHash42("apple")
b = AlwaysHash42("banana")

print("Hash of a:", hash(a))
print("Hash of b:", hash(b))

# Put them into a dictionary
d = {a: "fruit A", b: "fruit B"}

print("Dictionary:", d)
print("Lookup a:", d[a])
print("Lookup b:", d[b])
```

### Output:

```
Hash of a: 42
Hash of b: 42
Dictionary: {Obj(apple): 'fruit A', Obj(banana): 'fruit B'}
Lookup a: fruit A
Lookup b: fruit B
```

---

✅ Explanation:

* Both `a` and `b` hash to the same number (`42`).
* But Python’s dictionary doesn’t break:

  * It first uses the hash to find a bucket.
  * If multiple keys share that bucket (collision), it uses `__eq__` to tell them apart.

That’s why a hash function doesn’t need to be **one-to-one**.

